In [447]:
import requests
from bs4 import BeautifulSoup
import urllib

In [450]:
# 1~ 513 페이지를 네비게이션하면서 1996년도에서 오늘까지의 일일 종가 데이터를 가져온다.
domain = 'http://finance.naver.com/item/sise_day.nhn?code=005930&page='

In [451]:
list = []

#for page in range(1,513) :
for page in range(1,5) :
    url = domain + str(page)
    source_code = requests.get(url, timeout=None)
    soup = BeautifulSoup(source_code.text,  'lxml')
    tableSoup = soup.find('table', attrs={'class', 'type2'} )
    trSoup = tableSoup.find_all('tr')
    i = 0
    rowcount =0;
    
    for tr in  trSoup :
      i = i + 1
      if i > 2  and i < 16 and i <= len(tr) :
        if i == 8 or i == 9 or i == 10 :
            continue
        j = 0           
        dic = {}

        for td in tr :
            
            j = j + 1
            if j == 2 :
                date = str(td.span.text)
                dic['date'] = date
            if j == 4 :
                price = str(td.span.text)
                dic['price'] = int(price.replace(',',''))
            if j == 6 :
                change = str(td.span.text).strip()
                if change != '0' :
                    if td.img['src'].endswith('ico_up.gif') or td.img['src'].endswith('ico_up2.gif') :
                        dic['change'] = int(change.replace(',',''))
                    else :
                        dic['change'] = -1 * int(change.replace(',',''))
        list.append(dic)

In [452]:
len(list)

40

In [453]:
list[3]

{'change': 72000, 'date': '2016.10.06', 'price': 1691000}

In [454]:
# 하루 등락률이 10프로 하락하는 지점을 찾는다. 하락을 했다면 그날의 시가를 가지고 향후 언제 그 가격을 넘어서는지를 찾을 것이다.
# 전날 종가를 기준으로 등락률을 매기는게 맞는데 그냥 당일 종가와 등락 가격을 가지고 퍼센티지를 계산함.

fileter_list = []
for day in list :
    #print(day['date'] +  " : " + str(day['price']) + " : " + str(day['change']))
    percent = ( day['price'] + day['change'] ) / day['price']  * 100 - 100
    #print(day['date'] + " : " + str(percent))
    if percent < -5 :
        fileter_list.append(day)
        

In [455]:
# 그래서 우선 list를 filter해서 가지고 있고 그 시가를 가지고 있는 list를 만들 것이다. 그리고 이 filtered_list를 가지고 다시 원래의 list 
fileter_list

[{'change': -135000, 'date': '2016.10.11', 'price': 1545000},
 {'change': -110000, 'date': '2016.09.12', 'price': 1465000}]

In [458]:
# 그다음 이시기에 어떤 뉴스가 있었는지를 찾기 위해서 네이버 증권의 뉴스데이터를 이용한다.
news_domain = 'http://finance.naver.com/item/news_news.nhn?code=005930'
news_domain_with_pageNo = 'http://finance.naver.com/item/news_news.nhn?code=005930&page='

source_code = requests.get(news_domain, timeout=None)
soup = BeautifulSoup(source_code.text,  'lxml')

tableSoup = soup.find('table', attrs={'class', 'Nnavi'} )
td = tableSoup.find('td', attrs={'class','pgRR'}).a['href']
last_page_index = int(td[td.index('page=') + 5:])

In [459]:
new_list = []

#for page in range(1,last_page_index) :
for page in range(1,3) :
    url = news_domain_with_pageNo + str(page)
    source_code = requests.get(url, timeout=None)
    soup = BeautifulSoup(source_code.text,  'lxml')

    newsTabelSoup = soup.find('table', attrs={'class', 'type2'} )
    trSoup = newsTabelSoup.find_all('tr')

    i = 0
    rowcount =0;
    
    for tr in  trSoup :
        i = i + 1
        if i > 2  and i < 16 :
            if i == 8 or i == 9 :
                continue
            j = 0           
            dic = {}
        
            for td in tr :    
                j = j + 1
                if j == 2 :
                    dic['date'] = str(td.span.text)
                if j == 4 :
                    dic['title'] = td.a.text
                if j == 6 :
                    dic['publish'] = td.text
            new_list.append(dic)

In [460]:
new_list

[{'date': '2016.10.11 22:17',
  'publish': '파이낸셜뉴스',
  'title': '삼성, ‘아틱’ 앞세워 IoT 패권 잡는다'},
 {'date': '2016.10.11 21:54',
  'publish': '파이낸셜뉴스',
  'title': '[갤노트7 판매중단] 갤노트7 13일부터 갤S7으로 교...'},
 {'date': '2016.10.11 21:41',
  'publish': '파이낸셜뉴스',
  'title': '[갤노트7 판매중단] 갤노트7 사태 계열사로 확산.. ...'},
 {'date': '2016.10.11 20:11',
  'publish': '이데일리',
  'title': '[그래픽]주가 추이로 보는 갤노트7 출시부터 단종까지'},
 {'date': '2016.10.11 20:09',
  'publish': '아시아경제',
  'title': '갤럭시노트7 단종…출시 2개월만 "연말까지 교환·환불"...'},
 {'date': '2016.10.11 19:50',
  'publish': '이데일리',
  'title': '[일지]갤노트7 출시부터 단종까지'},
 {'date': '2016.10.11 19:01',
  'publish': '머니투데이',
  'title': "'단종' 갤노트7 소비자, 교환·환불은 어떻게?"},
 {'date': '2016.10.11 18:48',
  'publish': '이데일리',
  'title': "[갤노트7 단종]안전당국 발표 전 '결단'..삼성 위기..."},
 {'date': '2016.10.11 18:42',
  'publish': '헤럴드경제',
  'title': '삼성전자 갤노트7 단종 공식화…13일부터 제품 교환 환...'},
 {'date': '2016.10.11 18:38',
  'publish': '파이낸셜뉴스',
  'title': '갤노트7, 13일부터 교환-환불 시작.. 교환 시 차액...'},
 {},
 {'date': '2016.10.1